In [ ]:
from google.colab import drive
import json
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K


drive.mount("/content/drive")
proj_dir = "/content/drive/MyDrive/QA_project" #Change to directory where there are the train and dev set and glove embeddings

In [ ]:
!unzip "/content/drive/MyDrive/QA_project/glove.6B.zip" -d "/content"

In [ ]:
with open('/content/drive/MyDrive/QA_project/train-v2.0.json') as f:
  train = json.load(f)

with open('/content/drive/MyDrive/QA_project/dev-v2.0.json') as f:
  dev = json.load(f)

In [ ]:
# Load train data
train_contexts = []
train_questions = []
train_questions_id = [] #used for evaluation script
train_is_imp = []
train_answers = []
train_answers_start = []
train_answers_end = []
train_index = []

i = 0

for data in train['data']:
  title = data['title']

  for paragraph in data['paragraphs']:
    context = paragraph['context']

    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      is_impossible = qa['is_impossible']

      if not is_impossible:
        for answer in qa['answers']:
          text = answer['text']
          start = answer['answer_start']
          
          train_is_imp.append(0)
          train_contexts.append(context)
          train_questions.append(question)
          train_questions_id.append(id)
          train_answers.append(text)
          train_answers_start.append(start)
          train_answers_end.append(start + len(text))
          train_index.append(i)
          i+=1

In [ ]:
# Load dev data
dev_contexts = []
dev_questions = []
dev_questions_id = [] #used for evaluation script
dev_is_imp = []
dev_answers = []
dev_answers_start = []
dev_answers_end = []

for data in dev['data']:
  title = data['title']

  for paragraph in data['paragraphs']:
    context = paragraph['context']

    for qas in paragraph['qas']:
      question = qas['question']
      id = qas['id']
      is_impossible = qas['is_impossible']

      if not is_impossible:
        for answer in qas['answers']:
          text = answer['text']
          start = answer['answer_start']

          dev_is_imp.append(0)
          dev_contexts.append(context)
          dev_questions.append(question)
          dev_questions_id.append(id)
          dev_answers.append(text)
          dev_answers_start.append(start)
          dev_answers_end.append(start + len(text))

In [ ]:
# Tokenizer
from nltk.tokenize.regexp import RegexpTokenizer

tkn = RegexpTokenizer('[\s!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n—]', gaps=True)

In [ ]:
# Tokenize training set and build vocabulary
vocab = set()

tokenized_train_contexts = tkn.tokenize_sents(map(lambda x:x.lower(),train_contexts))
span_train_contexts = list(tkn.span_tokenize_sents(map(lambda x:x.lower(),train_contexts)))

tokenized_train_questions = tkn.tokenize_sents(map(lambda x:x.lower(),train_questions))

for item in tokenized_train_contexts:
  vocab.update(item)

for item in tokenized_train_questions:
  vocab.update(item)

In [ ]:
# Tokenize dev set
tokenized_dev_contexts = tkn.tokenize_sents(map(lambda x:x.lower(),dev_contexts))
span_dev_contexts = list(tkn.span_tokenize_sents(map(lambda x:x.lower(),dev_contexts)))

tokenized_dev_questions = tkn.tokenize_sents(map(lambda x:x.lower(),dev_questions))

In [ ]:
# Build word index
word_index = {}
for idx, voc in enumerate(vocab):
  word_index[voc] = idx + 2

In [ ]:
# Build token-answer correspondence training
train_start = []
train_end = []

for answ_start, answ_end, span_context, is_imp in zip(train_answers_start, train_answers_end, span_train_contexts, train_is_imp):
  if is_imp == 1:
    train_start.append([0]*len(span_context))
    train_end.append([0]*len(span_context))
  else:
    answer_enc = []
    answer_start_enc = []
    answer_end_enc = [0]*len(span_context)

    started = False
    end_idx = None

    for idx, span in enumerate(span_context):
      if span[0] >= answ_start and not started:
        answer_start_enc.append(1)
        answer_enc.append(1)
        started = True
        end_idx = idx
      elif started and span[1] <= answ_end:
        end_idx = idx
        answer_start_enc.append(0)
        answer_enc.append(1)
      else:
        answer_start_enc.append(0)
        answer_enc.append(0)

    if end_idx:
      answer_end_enc[end_idx] = 1

    train_start.append(answer_start_enc)
    train_end.append(answer_end_enc)

In [ ]:
# Build token-answer correspondence dev
dev_start = []
dev_end = []

for answ_start, answ_end, span_context, is_imp in zip(dev_answers_start, dev_answers_end, span_dev_contexts, dev_is_imp):
  if is_imp == 1:
    dev_start.append([0]*len(span_context))
    dev_end.append([0]*len(span_context))
  else:
    answer_enc = []
    answer_start_enc = []
    answer_end_enc = [0]*len(span_context)

    started = False
    end_idx = None

    for idx, span in enumerate(span_context):
      if span[0] >= answ_start and not started:
        answer_start_enc.append(1)
        answer_enc.append(1)
        started = True
        end_idx = idx
      elif started and span[1] <= answ_end:
        end_idx = idx
        answer_start_enc.append(0)
        answer_enc.append(1)
      else:
        answer_start_enc.append(0)
        answer_enc.append(0)

    if end_idx:
      answer_end_enc[end_idx] = 1

    dev_start.append(answer_start_enc)
    dev_end.append(answer_end_enc)

In [ ]:
del span_train_contexts
del span_dev_contexts

In [ ]:
def tokenized_texts_to_sequences(tkn_texts, word_index):
  sequences = []
  for text in tkn_texts:
    seq = []
    for word in text:
      seq.append(word_index[word]) if word in word_index.keys() else seq.append(1)
    sequences.append(seq)
  
  return sequences

# Integer encoding training
train_ctx = tokenized_texts_to_sequences(tokenized_train_contexts, word_index)
train_q = tokenized_texts_to_sequences(tokenized_train_questions, word_index)

# Integer encoding dev
dev_ctx = tokenized_texts_to_sequences(tokenized_dev_contexts, word_index)
dev_q = tokenized_texts_to_sequences(tokenized_dev_questions, word_index)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Pad train sequences and answer encoding
train_ctx = pad_sequences(train_ctx, padding='post')
train_q = pad_sequences(train_q, padding='post')
train_start = pad_sequences(train_start, padding='post')
train_end = pad_sequences(train_end, padding='post')

# Pad dev sequences
dev_ctx = pad_sequences(dev_ctx, padding='post')
dev_q = pad_sequences(dev_q, padding='post')
dev_start = pad_sequences(dev_start, padding='post')
dev_end = pad_sequences(dev_end, padding='post')

In [ ]:
# Save target list to numpy array
train_is_imp_arr = np.array(train_is_imp)

dev_is_imp_arr = np.array(dev_is_imp)

In [ ]:
# shuffle train data
import random

random.seed(42)
shuffler = np.random.permutation(train_ctx.shape[0])
train_ctx_shuffled = train_ctx[shuffler]
train_q_shuffled = train_q[shuffler]
train_start_shuffled = train_start[shuffler]
train_end_shuffled = train_end[shuffler]
train_imp_shuffled = train_is_imp_arr[shuffler]

train_index = np.array(train_index)
train_index_shuffled = train_index[shuffler]

In [ ]:
# Preparing embedding
embeddings_index = {}
with open('/content/glove.6B.300d.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [ ]:
# Define embedding matrix
embedding_matrix = np.zeros((len(word_index) + 2, 300))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
num_words = len(word_index) + 2

MODEL DEFINITION

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import *
import keras.backend as K

In [ ]:
class RepeatConcatMult(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(RepeatConcatMult, self).__init__()

  def compute_mask(self, inputs, mask=None):
    if mask:
      return mask[0]
    return mask

  def call(self, inputs, training, mask=None):
    ctx = inputs[0]
    q = inputs[1]

    q_repeated = K.tile(q, [1,K.shape(ctx)[1],1])

    return K.concatenate([K.concatenate([q_repeated, ctx], axis=-1), tf.multiply(q_repeated,ctx)])

  def get_config(self):
    config = super().get_config().copy()
        
    return config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [ ]:
# define input layers
question_input = keras.Input(shape=(None,), name="question")
context_input = keras.Input(shape=(None,), name="context")

# embedding
token_emb = Embedding(num_words, 300, weights=[embedding_matrix], trainable=False, mask_zero=True)
q_emb = token_emb(question_input)
q_emb = Dropout(0.4)(q_emb)
c_emb = token_emb(context_input)
c_emb = Dropout(0.4)(c_emb)

# encoder
q_encoding = Bidirectional(GRU(units=64, return_sequences=True, dropout=0.1))(q_emb)
c_encoding = Bidirectional(GRU(units=64, return_sequences=True, dropout=0.1))(c_emb)

# combination
q_pool = GlobalAveragePooling1D(keepdims=True)(q_encoding)
concat = RepeatConcatMult()([c_encoding, q_pool])

# prediction
dense = TimeDistributed(Dense(100,activation='relu'))(concat)
dense = Dropout(0.2)(dense)
dense = TimeDistributed(Dense(50,activation='relu'))(dense)
dense = Dropout(0.2)(dense)

start_predictor = TimeDistributed(Dense(10,activation='relu'))(dense)
start_predictor = Dropout(0.1)(start_predictor)
start_predictor = TimeDistributed(Dense(units=1, activation='linear'))(start_predictor)
start_predictor = Reshape((-1,))(start_predictor)
start_predictor = Softmax(axis=-1, name="start")(start_predictor)

end_predictor = TimeDistributed(Dense(10,activation='relu'))(dense)
end_predictor = Dropout(0.1)(end_predictor)
end_predictor = TimeDistributed(Dense(units=1, activation='linear'))(end_predictor)
end_predictor = Reshape((-1,))(end_predictor)
end_predictor = Softmax(axis=-1, name="end")(end_predictor)

model = keras.Model(
    inputs=[question_input, context_input],
    outputs=[start_predictor, end_predictor]
)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss={"start": keras.losses.CategoricalCrossentropy(),
          "end": keras.losses.CategoricalCrossentropy()
    }
)

In [ ]:
keras.utils.plot_model(model)

In [ ]:
model.summary()

In [ ]:
# apply early stopping
import time 

logdir = os.path.join(os.curdir, "logs", "run_{}".format(time.time()))

callbacks = [keras.callbacks.TensorBoard(logdir),
             keras.callbacks.EarlyStopping(patience=5),
             keras.callbacks.ModelCheckpoint("qa_attention_model.h5",save_best_only=True)]

In [ ]:
max = 86821
history = model.fit(x={"context": train_ctx_shuffled, "question": train_q_shuffled},
                   y={"start":train_start_shuffled, "end": train_end_shuffled},
                   epochs=30,
                   batch_size=512,
                   validation_split=0.2,
                   callbacks=callbacks
)

In [ ]:
prediction = model.predict([dev_q, dev_ctx])

In [ ]:
def answer(start, end, tokens):
  return tokens[start:end + 1]

In [ ]:
preds = {}
preds_list = {}
for idx in range(len(prediction[0])):
  example = prediction[0][idx]
  pred_start = np.argmax(example)

  example = prediction[1][idx]
  pred_end = np.argmax(example)

  pred_ans = answer(pred_start, pred_end, tokenized_dev_contexts[idx])

  id = dev_questions_id[idx]
  preds[id] = ' '.join(pred_ans)
  preds_list[id] = pred_ans

with open('/content/drive/MyDrive/QA_project/preds.json', 'w') as f:
  json.dump(preds, f)

with open('/content/drive/MyDrive/QA_project/preds_list.json', 'w') as f:
  json.dump(preds_list, f)

In [ ]:
np.save('/content/drive/MyDrive/QA_project/start_token.npy', prediction[0])
np.save('/content/drive/MyDrive/QA_project/end_token.npy', prediction[1])
np.save('/content/drive/MyDrive/QA_project/scores.npy', prediction[2])